In [ ]:
!pip install langchain langchain-openai langchain-community faiss-cpu unstructured

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
import os
import random # ランダムな順番を生成するためにインポート
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader # TextLoaderを追加
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from google.colab import drive, userdata

# ==============================================================================
# 1. Characterクラスの定義（RAG学習ロジックを含む）
# ==============================================================================
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str):
        self.name = name
        # drive.mount("/content/drive")
        print(f"キャラクター「{self.name}」を構築中...")
        # private_docs_path = os.path.join("/content/drive/My Drive/",private_docs_path)
        # shared_logs_path = os.path.join("/content/drive/My Drive/",shared_logs_path)
        # --- ドキュメント読み込み処理の改良 ---
        # 1. 個別の知識ファイルを読み込む
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt")
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")

        # 2. 共有の会話ログから、自分に関係のあるものだけを読み込む
        shared_docs = []
        for filename in os.listdir(shared_logs_path):
            log_path = os.path.join(shared_logs_path, filename)
            log_loader = TextLoader(log_path)
            shared_docs.extend(log_loader.load())
        print(f"  > 関連する会話ログを {len(shared_docs)} 件読み込みました。")

        # 3. 全てのドキュメントを結合
        all_docs = private_docs + shared_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        # --- ここまでが改良箇所 ---

        # LLMとEmbeddingモデル
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()

        # 分割とベクトル化
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()

        # RAGチェーンの構築（変更なし）
        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")

    def speak(self, input_text: str, chat_history: list):
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ==============================================================================
# 2. DialogueManagerクラスの定義（対話フローを改良）
# ==============================================================================
# ==============================================================================
# 2. DialogueManagerクラスの定義（対話フローを改良）
# ==============================================================================
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        self.shared_history.append(HumanMessage(content=f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"))
        # 最初の入力は、全キャラクターへの問いかけとする
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    # 【改善点2】対話フローを「キャッチボール形式」に大幅改善
    def run_discussion(self, turns_per_character: int = 5):
        """
        全テーマにわたる対談を実行する。
        直前の発言者とは別のキャラクターが応答することで、自然な対話を実現する。
        """
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None # 直前の発言者を記録する変数

            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")

            for turn in range(total_turns):
                # 次の発言者を決定する
                # 直前の発言者がいる場合は、そのキャラクターを除いたリストからランダムに選ぶ
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                    speaker = random.choice(possible_speakers)
                else:
                    # 最初の発言者は全員からランダムに選ぶ
                    speaker = random.choice(self.characters)

                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")

                response = speaker.speak(current_input, self.shared_history)

                # 念のため、ここでも文字数制限をかける（プロンプトの指示が最も重要）
                response = response[:200]

                print(response)
                print("-" * 20 + "\n")

                # 共有履歴と次の入力を更新
                self.shared_history.append(AIMessage(content=response, name=speaker.name))
                current_input = response
                last_speaker = speaker # 直前の発言者を更新

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # 【改善点1】プロンプトに文字数制限の指示を追加
    character_definitions = [
        {"name": "忍者", "private_path": "./忍者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、どこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},
        {"name": "侍", "private_path": "./侍/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、冷静かつ客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"},
        {"name": "記者", "private_path": "./記者/", "shared_path": "./conversation_logs/", "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"}
    ]

    discussion_topics = [
        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",
        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",
        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"
    ]

    # --- 実行 ---
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"]
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    # 【改善点3】各テーマで、各キャラクターが5回ずつ話すように設定
    manager.run_discussion(turns_per_character=5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
キャラクター「忍者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 関連する会話ログを 1 件読み込みました。
  > 合計 2 件のドキュメントで知識ベースを構築します。
キャラクター「忍者」の構築完了。
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
キャラクター「侍」を構築中...
  > 個別知識を 1 件読み込みました。
  > 関連する会話ログを 1 件読み込みました。
  > 合計 2 件のドキュメントで知識ベースを構築します。
キャラクター「侍」の構築完了。
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
キャラクター「記者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 関連する会話ログを 1 件読み込みました。
  > 合計 2 件のドキュメントで知識ベースを構築します。
キャラクター「記者」の構築完了。

【新たなテーマ】: 名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？

今回のテーマでは、合計 15 回の発言が予定されています。
--------------------------------------------------

--- (ターン1/15) 忍者の発言 ---
忍者: 「まずは、拙者の意見を申し上げるでござる。2025年度前半は、勝点の積み上げが順調で、特に強い相手にも善戦した試合が多かったでござる。チームの連携も良く、戦術が浸透している印象があるでござるよﾆﾝﾆﾝ。残留争いを意識し